In [1]:
import sys
import os
from prefect import task, Flow

abspath = os.path.dirname(os.path.normpath(os.path.abspath(os.path.dirname(''))))

sys.path.append(abspath)

driver_path = os.path.join(abspath, 'drivers', 'chromedriver.exe')

raw_download_path = os.path.normpath(os.path.join(abspath, 'data', 'raw'))

from toolbox.scrapers import *

In [2]:
hospital_urls = get_source_urls(driver_path, raw_download_path)


C:\Users\remot\OneDrive\Desktop\Personal\nc-hospital-transparency\hospitals.csv
                            hospital_name  \
0                Duke University Hospital   
1         North Carolina Baptist Hospital   
2   University of North Carolina Hospital   
3  Appalachian Regional Healthcare System   
4                           Atrium Health   

                             hospital_id  \
0               duke-university-hospital   
1        north-carolina-baptist-hospital   
2  university-of-north-carolina-hospital   
3             app-regional-health-system   
4                          atrium-health   

                                        hospital_url  hospital_prefix  
0  https://www.dukehealth.org/paying-for-care/wha...              NaN  
1  https://www.wakehealth.edu/Patient-and-Family-...              NaN  
2  https://www.unchealthcare.org/about-us/billing...              NaN  
3  https://apprhs.org/estimate-your-healthcare-cost/              NaN  
4  https://atriumhealth.

In [103]:
def get_nhrmc(hospital_id: str, hospital_urls: dict, raw_download_path: str) -> pd.DataFrame:

    """Get New Hanover Regional Medical Center data and download only the CSV file"""
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


    url_list = hospital_urls[hospital_id]

    download_path = os.path.join(raw_download_path, hospital_id)
    
    create_directory(download_path)

    ip_df_list = []

    op_df_list = []

    for url in url_list:
         
        # infer filename from url 
        filename = url.split('/')[-1]
        # download the file

        print(filename)

        response = requests.get(url, headers=headers)

        # write reponse to csv file from excel

        if 'new-hanover' in filename:

            df_ip = pd.read_excel(response.content,  sheet_name=3, skiprows=5)

            df_op = pd.read_excel(response.content, sheet_name=4, skiprows=5)

            df_ip.to_csv(os.path.join(download_path, filename.replace('.xlsx', 'ip.csv')), index=False)
    
            df_op.to_csv(os.path.join(download_path, filename.replace('.xlsx', 'op.csv')), index=False)

            df_ip['Hosp'] = 'New Hanover'

            df_op['Hosp'] = 'New Hanover'
            
            ip_df_list.append(df_ip)

            op_df_list.append(df_op)


        if 'pender' in filename:

            df_ip = pd.read_excel(response.content,  sheet_name=3, skiprows=4)

            df_op = pd.read_excel(response.content, sheet_name=4, skiprows=5)

            df_ip.to_csv(os.path.join(download_path, filename.replace('.xlsx', '_ip.csv')), index=False)
    
            df_op.to_csv(os.path.join(download_path, filename.replace('.xlsx', '_op.csv')), index=False)

            df_ip['Hosp'] = 'Pender'

            df_op['Hosp'] = 'Pender'

            ip_df_list.append(df_ip)

            op_df_list.append(df_op)

    return pd.concat(ip_df_list), pd.concat(op_df_list)
    


In [104]:
op_df, ip_df = get_nhrmc('nhrmc-health', hospital_urls, raw_download_path)


340141-new-hanover-regional-medical-center-standard-charges.xlsx
341307-pender-memorial-hospital-standard-charges.xlsx


In [96]:
a

,APC,Description,Aetna,Atlantic Packaging,BCBS HMO PPO,Cigna PPO,Multiplan,UHC HMO,UHC NHRMC Employee Rates,BCBS Blue Value,...,Medcost Columbus,De-Identified Minimum Negotiated Charge,De-Identified Maximum Negotiated Charge,APC Average Charge,Aetna HMO/PPO,BCBS PPO/HMO,Cigna HMO/PPO,Humana PPO,MedCost PPO,UHC HMO/PPO
0,5012,Clinic Visits and Related Services,260.0,211.0,131.0,177.0,689.0,47.0,275.0,NaN,...,NaN,47,689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5021,Level 1 Type A ED Visits,262.0,NaN,148.0,201.0,271.0,312.0,121.0,133.0,...,NaN,121,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5022,Level 2 Type A ED Visits,434.0,258.0,244.0,333.0,449.0,517.0,201.0,220.0,...,NaN,201,843,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5023,Level 3 Type A ED Visits,1471.0,833.0,803.0,1075.0,1538.0,1420.0,653.0,711.0,...,1057.0,653,1664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5024,Level 4 Type A ED Visits,3703.0,1768.0,2072.0,2971.0,3874.0,1932.0,1797.0,1935.0,...,3728.0,1645,3874,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,5732,Level 2 Minor Procedures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,410,858,1059.190000,816.0,410.0,505.0,858.0,794.0,744.0
38,5733,Level 3 Minor Procedures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,110,230,283.799630,219.0,110.0,135.0,230.0,213.0,199.0
39,5734,Level 4 Minor Procedures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,404,845,1043.169500,803.0,404.0,498.0,845.0,782.0,732.0
40,5735,Level 5 Minor Procedures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,362,758,936.385000,721.0,362.0,447.0,758.0,702.0,657.0
